In [301]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [302]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/"

In [303]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']
columns = ['row_path_1','row_path_2','row_path_3','row_path_4', 'row_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [304]:
def file_paths(df):
    # Create different colums for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    mark = ".json"

    df['path_2'] = df['path_2'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_3'] = df['path_3'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_4'] = df['path_4'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [305]:
def string_to_dict(s):
    if ',' not in s and ':' not in s:
        return s
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [306]:
def row_column_paths(df):
# Initialize the new columns
    df['col_path'] = '' 
    df['row_path_1'] = ''  
    df['row_path_2'] = ''  
    df['row_path_3'] = ''  

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'col_path'] = list(row['value'].keys())
        
    df['col_path'] = df['col_path'].astype('str').apply(eval).str[0]

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('col_path')


    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 
    df['row_path_1_values'] = df.apply(lambda row: [row['value'].get(row['col_path'], None)], axis=1)
    


    # Unlist and store in individual rows
    df['row_path_1_values'] = df['row_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_1_values'] = df['row_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    
    """
    # Convert string representations of dict or list to actual dict/list 
    df['row_path_1_values'] = df['row_path_1_values'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith(('{', '[')) else x
    )
    """

    # Assign row_path_1_values to row_path_1
    df['row_path_1'] = df['row_path_1_values']
    df = df.explode('row_path_1')


    df['row_path_1_values'] = df['row_path_1_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)

    # For the level 2 keys stored in row_path_1, extract the level 3 keys and store in row_path_2 
    #df['row_path_2_values'] = df.apply(lambda row: [row['row_path_1_values'].get(row['row_path_1'], None)] if isinstance(row['row_path_1_values'], dict) else None, axis=1)
    df['row_path_2_values'] = df.apply(lambda row: [row['row_path_1_values'].get(row['row_path_1'], None)]if isinstance (row['row_path_1_values'], dict) else None, axis=1)
    
    
    # Unlist and store in individual rows
    df['row_path_2_values'] = df['row_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_2_values'] = df['row_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    """
    # Convert string representations of dict or list to actual dict/list (if applicable)
    df['row_path_2_values'] = df['row_path_2_values'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith(('{', '[')) else x
    )
    """
    

    # Assign row_path_2_values to row_path_2
    df['row_path_2'] = df['row_path_2_values']
    df = df.explode('row_path_2')

    df['row_path_2_values'] = df['row_path_2_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    

    #ROW_PATH_3
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['row_path_3_values'] = df.apply(lambda row: [row['row_path_2_values'].get(row['row_path_2'], None)] if isinstance (row['row_path_2_values'], dict) else None, axis=1)

    # Unlist and store in individual rows
    df['row_path_3_values'] = df['row_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_3_values'] = df['row_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    df['row_path_3_values'] = df['row_path_3_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['row_path_3'] = df['row_path_3_values']
    df = df.explode('row_path_3')



    
    #ROW_PATH_4
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['row_path_4_values'] = df.apply(lambda row: [row['row_path_3_values'].get(row['row_path_3'], None)]if isinstance (row['row_path_3_values'], dict) else None, axis=1)
    
    df['row_path_4_values'] = df['row_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_4_values'] = df['row_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df[['row_path_3', 'row_path_4_values']])
    
    df['row_path_4_values'] = df['row_path_4_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    display(df[['row_path_3', 'row_path_4_values']])
    
    # Assign row_path_3_values to row_path_3
    df['row_path_4'] = df['row_path_4_values']
    df = df.explode('row_path_4')


    #ROW_PATH_5
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['row_path_5_values'] = df.apply(lambda row: [row['row_path_4_values'].get(row['row_path_4'], None)]if isinstance (row['row_path_4_values'], dict) else None, axis=1)
    
    df['row_path_5_values'] = df['row_path_5_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['row_path_5_values'] = df['row_path_5_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df[['row_path_4', 'row_path_5_values']])
    
    df['row_path_5_values'] = df['row_path_5_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    # Assign row_path_3_values to row_path_3
    df['row_path_5'] = df['row_path_5_values']
    df = df.explode('row_path_5')
    
   

    return df


In [307]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'col_path', 'row_path_1','row_path_2','row_path_3', 'row_path_4', 'row_path_5','data_type', 'row_path_1_values','row_path_2_values','row_path_3_values', 'row_path_4_values', 'row_path_5_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}Instagram/Output/Output_" + file_name + '.csv', index=False)

    return df

In [308]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)

    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)

    # Delete user specific informations
    df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [309]:
input_directory = Path(f'{main_path}Instagram/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input


In [310]:

for file in input_directory.iterdir():  
    if file.is_file():  
        print(file)
        structure_donations(file)
        

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/08-04-2025-json_structure instagram.json


,row_path_3,row_path_4_values
0,None,None
0,None,None
0,string,None
0,string,None
0,number,None
...,...,...
34,value,string
34,timestamp,number
34,href,string
34,value,string


,row_path_3,row_path_4_values
0,None,None
0,None,None
0,string,None
0,string,None
0,number,None
...,...,...
34,value,string
34,timestamp,number
34,href,string
34,value,string


,row_path_4,row_path_5_values
0,None,None
0,None,None
0,None,None
0,None,None
0,None,None
...,...,...
34,string,None
34,number,None
34,string,None
34,string,None


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/IG_json_structure_CASAS.json


...

,row_path_4,row_path_5_values
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
...,...,...
369,None,None
370,None,None
371,None,None
372,None,None


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/nv_insta.json


,row_path_3,row_path_4_values
0,None,None
0,None,None
0,string,None
0,string,None
0,number,None
...,...,...
97,None,None
97,None,None
97,None,None
97,None,None


,row_path_3,row_path_4_values
0,None,None
0,None,None
0,string,None
0,string,None
0,number,None
...,...,...
97,None,None
97,None,None
97,None,None
97,None,None


,row_path_4,row_path_5_values
0,None,None
0,None,None
0,None,None
0,None,None
0,None,None
...,...,...
97,None,None
97,None,None
97,None,None
97,None,None


In [311]:
# Path to the folder containing CSV files
output_path = f"{main_path}Instagram/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'col_path', 'row_path_1','row_path_2','row_path_3', 'row_path_4', 'row_path_5','data_type', 'row_path_1_values','row_path_2_values','row_path_3_values', 'row_path_4_values', 'row_path_5_values']


merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest  #
    merged_df = merged_df.merge(df, on=common_columns, how="outer")




# Filter where col1 contains 'messages', then drop duplicates based on col2
df_filtered = merged_df[merged_df["path_1"] == "messages"].drop_duplicates(subset="path_2")


# Append rows where col1 does not contain 'messages'
df_final = pd.concat([df_filtered, merged_df[merged_df["path_1"] != "messages"]], ignore_index=True)
df_final = df_final.replace('Missing', np.nan)


print(df_final.isna().sum())

# Save the final merged DataFrame
df_final.to_csv(f"{main_path}Instagram/Final/Merged_structures_IG.csv", index=False)

variable                0
value                   0
path_1                  0
path_2                967
path_3               1351
path_4               1415
json_name               0
col_path                0
row_path_1            130
row_path_2            594
row_path_3            763
row_path_4           1370
row_path_5           1406
data_type              61
row_path_1_values       0
row_path_2_values     130
row_path_3_values     594
row_path_4_values     763
row_path_5_values    1370
dtype: int64


/tmp/ipykernel_9694/1030047021.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.replace('Missing', np.nan)
